In [1]:
import itertools
from ipywidgets import interact, Button, IntText, GridBox, Layout, VBox, HBox, HTML, Output, Label
from IPython.display import display
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row, column, layout
from bokeh.models import CategoricalColorMapper, ColumnDataSource, Text
from bokeh.plotting import figure, output_file
from grid import Grid
output_notebook()

Loading BokehJS ...

In [3]:
g = Grid(4, 4)

line_width = 10
line_color = '#B8AD9A'  # the same for plot outline and rectangles border

p = figure(tools='', title=f"Score: {g.score}", plot_width=400, plot_height=400, 
           x_range=(-0.5, 3.5), y_range=(3.5, -0.5))

# make the graph look more like a game board
line_width = 10
line_color = '#B8AD9A'
p.outline_line_width = line_width
p.outline_line_color = line_color
p.xgrid.grid_line_color = p.ygrid.grid_line_color = None
p.title.align = 'right'
p.title.text_font_size = '1.5em'
p.axis.visible = False


color_dict = {0: '#D7D7D7', 2: '#ECE3AF', 4: '#E0D1A1', 8: '#D3B486', 16: '#F0AB79', 32: '#F18231',
              64: '#F73B0C', 128: '#DAD083', 256: '#D6C01E', 512: '#D4E80F', 1024: '#A3D829',
              2048: '#30D55A', 4096: '#B356E5', 8192: '#37E5E7', 16384: '#5117EA', 32768: '#C13695',
              65536: '#468A9C', 131072: '#9C4657'}
coord_pairs = [i for i in itertools.product(range(4), range(4))]
x, y = zip(*coord_pairs)

def update_source():
    g.spawn()
    vals = [g.to_numpy()[c] for c in coord_pairs]
    tile_colors = [color_dict[v] for v in vals]
    text_colors = ['black' if v != 0 else '#D7D7D7' for v in vals]  # make zero values invisible
    return dict(x=y, y=x, vals=vals, tile_colors=tile_colors, text_colors=text_colors)

# initial state of the data source
source = ColumnDataSource(update_source())

# draw tiles
p.rect(x='x', y='y', source=source, width=1, height=1, line_width=line_width,
       line_color=line_color, fill_color='tile_colors')

# add text
p.text(x='x', y='y', source=source, text='vals', text_align='center', 
       text_color='text_colors')

# this updates the grid object based on the direction of the calling button, then recreates the datasource
def on_button_clicked(b):
    direction = b.description.lower()
    if g.slide(direction):  # if the move generated any change on the grid -> update source
        source.data = update_source()
        p.title.text = f"Score: {g.score}"
        push_notebook()
        if g.game_over():
            print("Game Over")
            print("Re-run this cell to start again.")
            
show(p, notebook_handle=True)

# create action buttons
btn_up  = Button(description='Up', layout=Layout(width='auto', grid_area='btn_up'))
btn_left = Button(description='Left', layout=Layout(width='auto', grid_area='btn_left'))
btn_right = Button(description='Right', layout=Layout(width='auto', grid_area='btn_right'))
btn_down  = Button(description='Down', layout=Layout(width='auto', grid_area='btn_down'))

# link buttons to function
for button in [btn_up, btn_right, btn_down, btn_left]:
    button.on_click(on_button_clicked)

# arrange controls in a grid box
gbox = GridBox(children=[btn_up, btn_left, btn_right, btn_down], layout=Layout(
            width='25%',
            grid_template_rows='auto',
            grid_template_columns='auto',
            grid_template_areas='''
            ". btn_up btn_up ."
            "btn_left btn_left btn_right btn_right"
            ". btn_down btn_down ."
            '''
        ))
display(gbox)

GridBox(children=(Button(description='Up', layout=Layout(grid_area='btn_up', width='auto'), style=ButtonStyle(…

Game Over
Re-run this cell to start again.
